# Cornershop's Data Science Test

## Dev: Mateus Broilo
mateus.brilo90@gmail.com

* Objectives: Fine Tuning Training and validation

# Summary

* Initials
    * Loading Libraries and Packages
    * Configurations
    * Global Variables
* Loading data
    * Prepared Dataset  
* Splitting Train/Test data: 80/20
* CatBoost
    * Fine-Tuning
        * Best Parameters
        * Feature Importance
* Results
* Save model

# Initials
* Loading Libraries and Packages
* Configuration
* Global Variables

## Loading Libraries and Packages

In [1]:
# general
import warnings
import time
import gc

#data
import pandas as pd
import numpy as np
from datetime import (datetime, date)
import geopy.distance

## visualization
import matplotlib.pyplot as plt
import seaborn as sns

## model
import joblib
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import *
from sklearn.model_selection import RandomizedSearchCV
import catboost as cb
from catboost import Pool

In [2]:
ls ..\data\

 O volume na unidade C nÃ£o tem nome.
 O NÃºmero de SÃ©rie do Volume Ã© 1210-43E2

 Pasta de C:\Users\mbroilo\Documents\Arbeit\Personal\projects\cornershop\data

01/04/2022  20:54    <DIR>          .
31/03/2022  18:23    <DIR>          ..
29/03/2022  20:02        64.304.021 20220329_merged_full.csv
29/03/2022  20:02        12.563.082 20220329_merged_predict.csv
29/03/2022  20:03        51.741.160 20220329_merged_train_test.csv
01/04/2022  20:49         1.125.788 20220331_prep_20220329_merged_train_test.csv
               4 arquivo(s)    129.734.051 bytes
               2 pasta(s)   143.172.710.400 bytes disponÃ­veis


## Configurations

In [3]:
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

rc={'font.size': 14, 'axes.labelsize': 14, 'legend.fontsize': 12, 
    'axes.titlesize': 24, 'xtick.labelsize': 14, 'ytick.labelsize': 12}

sns.set(rc=rc)
sns.set_palette("Set2")
sns.set_style("darkgrid")#sns.set_style("whitegrid")

## Global Variables

In [4]:
PATH_RAWDATA = "../raw_data/"
PATH_DATA = "../data/"
DATASET_PREP = "20220331_prep_20220329_merged_train_test.csv"
DATASET_VAL = "20220329_merged_predict.csv"
MODEL = 'CatBoostRegressor.sav'
DATE = time.strftime("%Y%m%d")
SEED = 42 # The answer to the ultimate question of life, the universe and everything is 42.

# Loading Data
* Prepared Dataset 

## Prepared Dataset 

In [5]:
df = pd.read_csv(PATH_DATA + DATASET_PREP, sep=',', index_col='order_id')
print(df.shape)
df.head()

(8000, 16)


,on_demand,day_of_week,total_minutes,pure_time,found_rate,picking_speed,accepted_rate,rating,distance,no_item,quantity,item,seniority_41dc7c9e385c4d2b6c1f7836973951bf,seniority_50e13ee63f086c2fe84229348bc91b5b,seniority_6c90661e6d2c7579f5ce337c3391dbb9,seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f
order_id,,,,,,,,,,,,,,,,
0004a3841c1eeb6c6e77585a941c21e0,True,friday,39.001464,20.300000,0.8473,2.30,0.920000,4.88,0.579554,8.0,8.000,4,1,0,0,0
0005a6ecbbde1e8d273f5577bcff2c9c,True,friday,41.164442,20.666667,0.8407,1.57,0.840000,4.72,0.989345,2.0,2.000,1,0,1,0,0
0007baeb6700fc203be2d1f1e11222d7,False,saturday,136.453089,15.000000,0.9012,2.50,1.000000,4.92,3.735978,39.0,39.000,22,1,0,0,0
0012195a6a8ca9ec308a3010eeea8ebc,False,saturday,72.853598,18.000000,0.8781,1.27,0.920000,4.88,1.226007,13.0,13.000,11,0,0,1,0
0013011fa72b498b9feb84f4e7104980,False,saturday,143.040919,15.000000,0.8301,3.12,0.857143,5.00,1.905397,65.0,64.636,44,0,1,0,0


# Splitting Train/Test data: 80/20

In [6]:
features = [
    'seniority_41dc7c9e385c4d2b6c1f7836973951bf',
    'seniority_50e13ee63f086c2fe84229348bc91b5b',
    'seniority_6c90661e6d2c7579f5ce337c3391dbb9',
    'seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f',
    'on_demand',
    #day_of_week,
    'pure_time',
    'found_rate',
    'picking_speed',
    'accepted_rate',
    'rating',
    'distance',
    'no_item',
    'quantity',
    'item'
]

label = ['total_minutes']

X = df[features]
y = df[label]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = SEED)

In [7]:
print("Train dataset shape: " + f"{X_train.shape}")
print("Test dataset shape: " + f"{X_test.shape}")
print(40*"-")
print(f"X_train + X_test dataset: {X_train.shape[0] + X_test.shape[0]}")
print(f"Total dataset: {df.shape[0]}")

Train dataset shape: (6400, 14)
Test dataset shape: (1600, 14)
----------------------------------------
X_train + X_test dataset: 8000
Total dataset: 8000


# CatBoost
* Fine-Tuning
    * Best Parameters
    * Feature Importance

In [8]:
model = cb.CatBoostRegressor()

{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 0.800000011920929, 'use_best_model': False, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.05489800125360489, 'score_function': 'Cosine', 'task_type': 'CPU', 'leaf_estimation_iterations': 1, 'bootstrap_type': 'MVS', 'max_leaves': 64}

## Fine-Tuning

In [9]:
%%time

params = {
    'learning_rate': uniform(0.05, 0.02), # default = 0.05
    'depth': randint(6,12), # default = 6
    'l2_leaf_reg': randint(0,6), # default = 3
}

randomized_search_result = model.randomized_search(
    params,
    X=X_train,
    y=y_train,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 83.6054020	test: 83.8964239	best: 83.8964239 (0)	total: 145ms	remaining: 2m 24s
1:	learn: 79.6629502	test: 79.9242161	best: 79.9242161 (1)	total: 149ms	remaining: 1m 14s
2:	learn: 75.9511574	test: 76.1320300	best: 76.1320300 (2)	total: 152ms	remaining: 50.6s
3:	learn: 72.4342080	test: 72.5531164	best: 72.5531164 (3)	total: 155ms	remaining: 38.7s
4:	learn: 69.1834887	test: 69.2418869	best: 69.2418869 (4)	total: 160ms	remaining: 31.8s
5:	learn: 66.0606617	test: 66.0616265	best: 66.0616265 (5)	total: 162ms	remaining: 26.9s
6:	learn: 63.2140466	test: 63.1743272	best: 63.1743272 (6)	total: 165ms	remaining: 23.4s
7:	learn: 60.4488023	test: 60.3677941	best: 60.3677941 (7)	total: 167ms	remaining: 20.7s
8:	learn: 57.9146647	test: 57.7895533	best: 57.7895533 (8)	total: 171ms	remaining: 18.8s
9:	learn: 55.4825135	test: 55.2659656	best: 55.2659656 (9)	total: 174ms	remaining: 17.2s
10:	learn: 53.2631679	test: 53.0135641	best: 53.0135641 (10)	total: 177ms	remaining: 15.9s
11:	learn: 51.224

105:	learn: 24.5973007	test: 23.8089317	best: 23.8089317 (105)	total: 491ms	remaining: 4.14s
106:	learn: 24.5903367	test: 23.8006484	best: 23.8006484 (106)	total: 494ms	remaining: 4.12s
107:	learn: 24.5752034	test: 23.7910907	best: 23.7910907 (107)	total: 499ms	remaining: 4.12s
108:	learn: 24.5643750	test: 23.7864967	best: 23.7864967 (108)	total: 502ms	remaining: 4.1s
109:	learn: 24.5351971	test: 23.7694532	best: 23.7694532 (109)	total: 505ms	remaining: 4.08s
110:	learn: 24.5167820	test: 23.7618713	best: 23.7618713 (110)	total: 508ms	remaining: 4.07s
111:	learn: 24.5045171	test: 23.7530703	best: 23.7530703 (111)	total: 510ms	remaining: 4.04s
112:	learn: 24.4915127	test: 23.7472374	best: 23.7472374 (112)	total: 516ms	remaining: 4.05s
113:	learn: 24.4832678	test: 23.7390225	best: 23.7390225 (113)	total: 521ms	remaining: 4.05s
114:	learn: 24.4642969	test: 23.7286769	best: 23.7286769 (114)	total: 524ms	remaining: 4.03s
115:	learn: 24.4577146	test: 23.7234322	best: 23.7234322 (115)	total: 5

196:	learn: 23.6344548	test: 23.4691044	best: 23.4661025 (191)	total: 845ms	remaining: 3.44s
197:	learn: 23.6232277	test: 23.4674703	best: 23.4661025 (191)	total: 849ms	remaining: 3.44s
198:	learn: 23.6136837	test: 23.4603095	best: 23.4603095 (198)	total: 852ms	remaining: 3.43s
199:	learn: 23.6039770	test: 23.4563693	best: 23.4563693 (199)	total: 855ms	remaining: 3.42s
200:	learn: 23.5910056	test: 23.4503601	best: 23.4503601 (200)	total: 863ms	remaining: 3.43s
201:	learn: 23.5873494	test: 23.4503702	best: 23.4503601 (200)	total: 866ms	remaining: 3.42s
202:	learn: 23.5849261	test: 23.4477840	best: 23.4477840 (202)	total: 868ms	remaining: 3.41s
203:	learn: 23.5791297	test: 23.4476627	best: 23.4476627 (203)	total: 872ms	remaining: 3.4s
204:	learn: 23.5727519	test: 23.4489787	best: 23.4476627 (203)	total: 877ms	remaining: 3.4s
205:	learn: 23.5618915	test: 23.4456465	best: 23.4456465 (205)	total: 880ms	remaining: 3.39s
206:	learn: 23.5556599	test: 23.4443133	best: 23.4443133 (206)	total: 88

304:	learn: 22.5923139	test: 23.3458352	best: 23.3458352 (304)	total: 1.39s	remaining: 3.18s
305:	learn: 22.5916567	test: 23.3458667	best: 23.3458352 (304)	total: 1.4s	remaining: 3.17s
306:	learn: 22.5863098	test: 23.3467349	best: 23.3458352 (304)	total: 1.41s	remaining: 3.17s
307:	learn: 22.5847422	test: 23.3471025	best: 23.3458352 (304)	total: 1.41s	remaining: 3.17s
308:	learn: 22.5740659	test: 23.3456480	best: 23.3456480 (308)	total: 1.41s	remaining: 3.16s
309:	learn: 22.5646888	test: 23.3426034	best: 23.3426034 (309)	total: 1.42s	remaining: 3.16s
310:	learn: 22.5565403	test: 23.3426371	best: 23.3426034 (309)	total: 1.43s	remaining: 3.16s
311:	learn: 22.5463911	test: 23.3395733	best: 23.3395733 (311)	total: 1.43s	remaining: 3.15s
312:	learn: 22.5399394	test: 23.3386413	best: 23.3386413 (312)	total: 1.43s	remaining: 3.15s
313:	learn: 22.5274119	test: 23.3322786	best: 23.3322786 (313)	total: 1.44s	remaining: 3.14s
314:	learn: 22.5186484	test: 23.3313211	best: 23.3313211 (314)	total: 1

...

459:	learn: 20.7349341	test: 24.4407968	best: 24.4276229 (326)	total: 2.15s	remaining: 2.52s
460:	learn: 20.7240394	test: 24.4437900	best: 24.4276229 (326)	total: 2.15s	remaining: 2.51s
461:	learn: 20.7142379	test: 24.4446095	best: 24.4276229 (326)	total: 2.15s	remaining: 2.5s
462:	learn: 20.7121297	test: 24.4451599	best: 24.4276229 (326)	total: 2.16s	remaining: 2.5s
463:	learn: 20.7062246	test: 24.4450494	best: 24.4276229 (326)	total: 2.17s	remaining: 2.51s
464:	learn: 20.7002265	test: 24.4457433	best: 24.4276229 (326)	total: 2.17s	remaining: 2.5s
465:	learn: 20.6975041	test: 24.4466871	best: 24.4276229 (326)	total: 2.18s	remaining: 2.49s
466:	learn: 20.6863442	test: 24.4532855	best: 24.4276229 (326)	total: 2.18s	remaining: 2.49s
467:	learn: 20.6802819	test: 24.4534159	best: 24.4276229 (326)	total: 2.19s	remaining: 2.48s
468:	learn: 20.6715416	test: 24.4527454	best: 24.4276229 (326)	total: 2.19s	remaining: 2.48s
469:	learn: 20.6545475	test: 24.4573023	best: 24.4276229 (326)	total: 2.2

579:	learn: 19.8609302	test: 24.4627889	best: 24.4276229 (326)	total: 2.67s	remaining: 1.94s
580:	learn: 19.8544056	test: 24.4631169	best: 24.4276229 (326)	total: 2.68s	remaining: 1.93s
581:	learn: 19.8470812	test: 24.4645033	best: 24.4276229 (326)	total: 2.68s	remaining: 1.93s
582:	learn: 19.8401608	test: 24.4618907	best: 24.4276229 (326)	total: 2.69s	remaining: 1.92s
583:	learn: 19.8389372	test: 24.4618446	best: 24.4276229 (326)	total: 2.69s	remaining: 1.92s
584:	learn: 19.8252150	test: 24.4625225	best: 24.4276229 (326)	total: 2.69s	remaining: 1.91s
585:	learn: 19.8192314	test: 24.4619471	best: 24.4276229 (326)	total: 2.7s	remaining: 1.91s
586:	learn: 19.8112873	test: 24.4634520	best: 24.4276229 (326)	total: 2.7s	remaining: 1.9s
587:	learn: 19.8028097	test: 24.4652379	best: 24.4276229 (326)	total: 2.7s	remaining: 1.89s
588:	learn: 19.7956912	test: 24.4636169	best: 24.4276229 (326)	total: 2.71s	remaining: 1.89s
589:	learn: 19.7864880	test: 24.4638020	best: 24.4276229 (326)	total: 2.71

698:	learn: 19.0166071	test: 24.4762550	best: 24.4276229 (326)	total: 3.21s	remaining: 1.38s
699:	learn: 19.0094992	test: 24.4766071	best: 24.4276229 (326)	total: 3.21s	remaining: 1.38s
700:	learn: 19.0008120	test: 24.4751147	best: 24.4276229 (326)	total: 3.21s	remaining: 1.37s
701:	learn: 18.9968377	test: 24.4764968	best: 24.4276229 (326)	total: 3.22s	remaining: 1.37s
702:	learn: 18.9937830	test: 24.4773034	best: 24.4276229 (326)	total: 3.22s	remaining: 1.36s
703:	learn: 18.9859898	test: 24.4777119	best: 24.4276229 (326)	total: 3.23s	remaining: 1.36s
704:	learn: 18.9756750	test: 24.4804286	best: 24.4276229 (326)	total: 3.23s	remaining: 1.35s
705:	learn: 18.9714798	test: 24.4761320	best: 24.4276229 (326)	total: 3.23s	remaining: 1.35s
706:	learn: 18.9637936	test: 24.4753592	best: 24.4276229 (326)	total: 3.24s	remaining: 1.34s
707:	learn: 18.9563329	test: 24.4753680	best: 24.4276229 (326)	total: 3.25s	remaining: 1.34s
708:	learn: 18.9469171	test: 24.4699555	best: 24.4276229 (326)	total: 

813:	learn: 18.2586107	test: 24.4708885	best: 24.4276229 (326)	total: 3.75s	remaining: 858ms
814:	learn: 18.2552406	test: 24.4733697	best: 24.4276229 (326)	total: 3.76s	remaining: 853ms
815:	learn: 18.2500625	test: 24.4749818	best: 24.4276229 (326)	total: 3.79s	remaining: 854ms
816:	learn: 18.2479715	test: 24.4756763	best: 24.4276229 (326)	total: 3.8s	remaining: 851ms
817:	learn: 18.2426884	test: 24.4735985	best: 24.4276229 (326)	total: 3.8s	remaining: 846ms
818:	learn: 18.2348332	test: 24.4746401	best: 24.4276229 (326)	total: 3.81s	remaining: 841ms
819:	learn: 18.2247697	test: 24.4750858	best: 24.4276229 (326)	total: 3.81s	remaining: 836ms
820:	learn: 18.2190384	test: 24.4786166	best: 24.4276229 (326)	total: 3.81s	remaining: 832ms
821:	learn: 18.2101296	test: 24.4810998	best: 24.4276229 (326)	total: 3.82s	remaining: 827ms
822:	learn: 18.2051028	test: 24.4820588	best: 24.4276229 (326)	total: 3.82s	remaining: 822ms
823:	learn: 18.2013428	test: 24.4838227	best: 24.4276229 (326)	total: 3.

925:	learn: 17.6140582	test: 24.5150336	best: 24.4276229 (326)	total: 4.46s	remaining: 357ms
926:	learn: 17.6096165	test: 24.5153848	best: 24.4276229 (326)	total: 4.47s	remaining: 352ms
927:	learn: 17.6056197	test: 24.5174141	best: 24.4276229 (326)	total: 4.49s	remaining: 348ms
928:	learn: 17.5992406	test: 24.5183255	best: 24.4276229 (326)	total: 4.49s	remaining: 343ms
929:	learn: 17.5901916	test: 24.5194420	best: 24.4276229 (326)	total: 4.5s	remaining: 339ms
930:	learn: 17.5879129	test: 24.5206153	best: 24.4276229 (326)	total: 4.5s	remaining: 334ms
931:	learn: 17.5857204	test: 24.5220394	best: 24.4276229 (326)	total: 4.5s	remaining: 329ms
932:	learn: 17.5810009	test: 24.5213118	best: 24.4276229 (326)	total: 4.51s	remaining: 324ms
933:	learn: 17.5769644	test: 24.5184615	best: 24.4276229 (326)	total: 4.51s	remaining: 319ms
934:	learn: 17.5700330	test: 24.5171471	best: 24.4276229 (326)	total: 4.52s	remaining: 314ms
935:	learn: 17.5667780	test: 24.5169412	best: 24.4276229 (326)	total: 4.5

### Best Parameters

In [10]:
print(model.get_params())

{'loss_function': 'RMSE', 'depth': 6.0, 'learning_rate': 0.05527980959709704, 'l2_leaf_reg': 4.0}


### Feature Importance

In [11]:
print("Feature importances:")

for i in range(df[features].columns.shape[0]):
    print(df[features].columns[i] + ": " + str(model.get_feature_importance(Pool(X_train, y_train))[i]))

Feature importances:
seniority_41dc7c9e385c4d2b6c1f7836973951bf: 0.6049398147199196
seniority_50e13ee63f086c2fe84229348bc91b5b: 0.3460985665606561
seniority_6c90661e6d2c7579f5ce337c3391dbb9: 1.1118759123179853
seniority_bb29b8d0d196b5db5a5350e5e3ae2b1f: 0.2379019756154152
on_demand: 5.884362522674268
pure_time: 7.740886066386296
found_rate: 10.413344629171604
picking_speed: 16.435622836987534
accepted_rate: 4.272565441070013
rating: 5.73387104624257
distance: 10.067344738790869
no_item: 5.243113762331128
quantity: 8.591850395510333
item: 23.31622229162141


In [ ]:
plt.barh(X_train.columns, model.get_feature_importance(Pool(X_train, y_train))

In [12]:
y_pred = model.predict(X_test)

# Results

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
ar2 = 1 - (1-r2_score(y_test, y_pred))*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
y_true = y_test.total_minutes.values.tolist()
print(f"RMSE: {rmse}")
print(f"R2: {r2}")
print(f"adjusted-R2: {ar2}" )
print(f"test deviance: {(2 * np.sum((y_true - y_pred) / y_pred - np.log(y_true) + np.log(y_pred)))}")

RMSE: 25.079017669465795
R2: 0.520889606633229
adjusted-R2: 0.5198390905005532
test deviance: 153.8720558763884


# Save Model

In [16]:
joblib.dump(model, DATE + '_' + MODEL) 

# And now to load...
#load = joblib.load(DATE + '_' + MODEL)